# Pytorch Implementation of EEGLearn - P. Bashivan
## Application: orientation decoding

This notebook describes a short summary of Pytorch implementation of the models described in "Learning Representations from EEG with Deep Recurrent-Convolutional Neural Networks." Bashivan et al. at International conference on learning representations (2016).

The rest of the code is in the different python scripts of this repo.

All the codes have been inspired from the [original github](https://github.com/pbashivan/EEGLearn).

Updates from LPSY lab (2023):
The networks had to be modified to match the new library version of pytorch. Also, many functions were adapted to our classification task.
In addition to that, we noticed that using a Selu instead of a Relu activation function would improve the results. This might result from the vanishing gradient problem encountered by the Relu. 

## Librairies Import

In [1]:
import numpy as np 
import scipy.io as sio
import torch
import os 
from os import path

import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader,random_split

from Utils import *
from Models import *

torch.manual_seed(1234)
np.random.seed(1234)

import warnings
warnings.simplefilter("ignore")

d:\antoine\documents\epfl_ma4\lab\dnn_sd\sd_dnn_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
participants = ['09','10','11','12']

In [3]:
for participant in participants:
    if not path.exists(f"Data_LPSY/{participant}_images_time.mat"):
        print(f"Time Windom Images didn't exist need to be created for participant {participant}")
        create_img(PATH="Data_LPSY", Nelectrodes=128, participant=participant)

## Loading the original Images 
The images have directly been taken from original implementation, given that they remain the same nevermind the implementation (Pytorch, Tensorflow, Theano).

In [4]:
# Mean_Images = sio.loadmat("Data_LPSY/images.mat")["img"] #corresponding to the images mean for all the seven windows
#print(np.shape(Mean_Images)) 
Images_t = np.vstack([np.transpose(sio.loadmat(f"Data_LPSY/{participant}_images_time.mat")["img"],(1,0,2,3,4)) for participant in participants])
#corresponding to the images mean for all the seven windows
print(np.shape(Images_t)) 
Orientation = np.concatenate([(sio.loadmat(f"Data_LPSY/{participant}_Orientations.mat")["orientation"]).reshape(-1) for participant in participants])
#corresponding to the signal label (i.e. load levels).
Label = (Orientation/20).astype(int)
print(np.shape(Label))

(3419, 5, 3, 32, 32)
(3419,)


In [5]:
np.unique(Label)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

## Select the "stripes" type only (in which orientation seems easier to decode)

In [6]:
stimType = np.concatenate([(sio.loadmat(f"Data_LPSY/{participant}_Labels.mat")["label"]).reshape(-1) for participant in participants])
Stripes_mask = stimType == 0
Images_t = Images_t[Stripes_mask]
Label = Label[Stripes_mask]

## Maxpool CNN
Build the Max-pooling model performing a maxpool over the 5 parallel convnets.

In [7]:
train_part = 0.8
test_part = 0.2

batch_size = 32

In [8]:
EEG = EEGImagesDataset(label=Label, image=Images_t)

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [9]:
print('Begin Training')
res = TrainTest_Model(MaxCNN, Trainloader, Testloader, n_epoch=45, learning_rate=0.00001, print_epoch=5, opti='Adam',
                     n_classes=9, n_window=5)

Begin Training
[5,  45]	loss: 2.193	Accuracy : 0.124		val-loss: 2.207	val-Accuracy : 0.079
[10,  45]	loss: 2.186	Accuracy : 0.138		val-loss: 2.209	val-Accuracy : 0.085
[15,  45]	loss: 2.175	Accuracy : 0.165		val-loss: 2.212	val-Accuracy : 0.082
[20,  45]	loss: 2.153	Accuracy : 0.179		val-loss: 2.223	val-Accuracy : 0.103
[25,  45]	loss: 2.120	Accuracy : 0.204		val-loss: 2.241	val-Accuracy : 0.106
[30,  45]	loss: 2.075	Accuracy : 0.230		val-loss: 2.263	val-Accuracy : 0.100
[35,  45]	loss: 2.015	Accuracy : 0.261		val-loss: 2.290	val-Accuracy : 0.091
[40,  45]	loss: 1.936	Accuracy : 0.303		val-loss: 2.316	val-Accuracy : 0.088
[45,  45]	loss: 1.835	Accuracy : 0.360		val-loss: 2.344	val-Accuracy : 0.094


## Temp CNN
FBuild the Conv1D model performing a convolution1D over the 5 parallel convnets.

In [10]:
print('Begin Training')
res = TrainTest_Model(TempCNN, Trainloader, Testloader, n_epoch=45, learning_rate=0.00001, print_epoch=5, opti='Adam',
                     n_classes=9, n_window=5)

Begin Training
[5,  45]	loss: 2.195	Accuracy : 0.118		val-loss: 2.202	val-Accuracy : 0.082
[10,  45]	loss: 2.191	Accuracy : 0.128		val-loss: 2.203	val-Accuracy : 0.082
[15,  45]	loss: 2.179	Accuracy : 0.217		val-loss: 2.205	val-Accuracy : 0.103
[20,  45]	loss: 2.112	Accuracy : 0.244		val-loss: 2.222	val-Accuracy : 0.106
[25,  45]	loss: 1.945	Accuracy : 0.311		val-loss: 2.299	val-Accuracy : 0.132
[30,  45]	loss: 1.759	Accuracy : 0.400		val-loss: 2.407	val-Accuracy : 0.126
[35,  45]	loss: 1.577	Accuracy : 0.464		val-loss: 2.530	val-Accuracy : 0.138
[40,  45]	loss: 1.386	Accuracy : 0.540		val-loss: 2.678	val-Accuracy : 0.129
[45,  45]	loss: 1.177	Accuracy : 0.632		val-loss: 2.877	val-Accuracy : 0.126


## LSTM CNN
Build the LSTM model applying a RNN over the 5 parallel convnets outputs

In [11]:
EEG = EEGImagesDataset(label=Label, image=Images_t)

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [12]:
print('Begin Training')
res = TrainTest_Model(LSTM, Trainloader, Testloader, n_epoch=45, learning_rate=0.00001, print_epoch=5, opti='Adam',
                     n_classes=9, n_window=5)

Begin Training
[5,  45]	loss: 2.192	Accuracy : 0.150		val-loss: 2.199	val-Accuracy : 0.132
[10,  45]	loss: 2.183	Accuracy : 0.194		val-loss: 2.199	val-Accuracy : 0.117
[15,  45]	loss: 2.167	Accuracy : 0.217		val-loss: 2.199	val-Accuracy : 0.114
[20,  45]	loss: 2.144	Accuracy : 0.227		val-loss: 2.201	val-Accuracy : 0.123
[25,  45]	loss: 2.114	Accuracy : 0.262		val-loss: 2.203	val-Accuracy : 0.126
[30,  45]	loss: 2.075	Accuracy : 0.291		val-loss: 2.209	val-Accuracy : 0.144
[35,  45]	loss: 2.026	Accuracy : 0.318		val-loss: 2.221	val-Accuracy : 0.138
[40,  45]	loss: 1.965	Accuracy : 0.355		val-loss: 2.241	val-Accuracy : 0.129
[45,  45]	loss: 1.888	Accuracy : 0.398		val-loss: 2.270	val-Accuracy : 0.117


## Mix CNN
Build the LSTM model applying a RNN and a CNN over the 5 parallel convnets outputs

In [13]:
EEG = EEGImagesDataset(label=Label, image=Images_t)

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [14]:
print('Begin Training')
res = TrainTest_Model(Mix, Trainloader, Testloader, n_epoch=60, learning_rate=0.00001, print_epoch=5, opti='Adam',
                     n_classes=9, n_window=5)

Begin Training
[5,  60]	loss: 2.193	Accuracy : 0.120		val-loss: 2.202	val-Accuracy : 0.091
[10,  60]	loss: 2.178	Accuracy : 0.200		val-loss: 2.204	val-Accuracy : 0.100
[15,  60]	loss: 2.114	Accuracy : 0.245		val-loss: 2.218	val-Accuracy : 0.106
[20,  60]	loss: 1.973	Accuracy : 0.295		val-loss: 2.277	val-Accuracy : 0.109
[25,  60]	loss: 1.801	Accuracy : 0.387		val-loss: 2.367	val-Accuracy : 0.117
[30,  60]	loss: 1.601	Accuracy : 0.469		val-loss: 2.485	val-Accuracy : 0.111
[35,  60]	loss: 1.354	Accuracy : 0.576		val-loss: 2.652	val-Accuracy : 0.138
[40,  60]	loss: 1.033	Accuracy : 0.702		val-loss: 2.903	val-Accuracy : 0.132
[45,  60]	loss: 0.674	Accuracy : 0.862		val-loss: 3.313	val-Accuracy : 0.123
[50,  60]	loss: 0.383	Accuracy : 0.955		val-loss: 3.847	val-Accuracy : 0.132
[55,  60]	loss: 0.203	Accuracy : 0.990		val-loss: 4.387	val-Accuracy : 0.117
[60,  60]	loss: 0.110	Accuracy : 0.999		val-loss: 4.901	val-Accuracy : 0.106
